<a href="https://colab.research.google.com/github/gustafvh/Sign-Language-ML-Interpreter/blob/master/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Must be run in Colab. Downloads dataset from github repo  
!rm -rf sample_data/
!mkdir output/
!git clone https://github.com/gustafvh/kex-dataset.git
!git clone https://github.com/gustafvh/SSL-Dataset.git


# Get dataset online example
# !wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
# !unzip -qq flower_data.zip

mkdir: cannot create directory ‘output/’: File exists
fatal: destination path 'kex-dataset' already exists and is not an empty directory.
fatal: destination path 'SSL-Dataset' already exists and is not an empty directory.


In [0]:
from PIL import Image
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from google.colab import files

import pandas as pd
import json
from skimage import transform
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet, InceptionV3, Xception, InceptionResNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing import image

# from kerastuner.tuners import RandomSearch
# from kerastuner.engine.hyperparameters import HyperParameters

# Time for 10 epochs with 2900 images:
# Locally CPU: 82 min (490 s per epoch)
# Colab GPU: 3 min (19 s per epoch)

from tensorflow.keras.utils import plot_model

import cv2
from google.colab.patches import cv2_imshow
import os

In [20]:
TRAINING_DATA_SET_PATH = '/content/SSL-Dataset/train'
VALIDATION_DATA_SET_PATH = '/content/SSL-Dataset/test'
TEST_DATA_SET_PATH = '/content/kex-dataset/test-images'
EPOCHS = 10  # 50-100 is a good range to try later
BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
DATASET_CATEGORIES = 29

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   fill_mode='nearest'
                                   )

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(TRAINING_DATA_SET_PATH,
                                                    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_SET_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    TEST_DATA_SET_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 14500 images belonging to 29 classes.
Found 2900 images belonging to 29 classes.
Found 1 images belonging to 1 classes.


In [0]:
def getPreTrainedModel():
    pre_trained_base_model = Xception(weights='imagenet',include_top=False,
                                         input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)
                                         )  # Remove last neuron layer (that contains 1000 neurons)

    my_model = pre_trained_base_model.output
    my_model = GlobalAveragePooling2D()(my_model)
    my_model = Dense(1024, activation='relu')(
        my_model)  # Add layer. Layer #1: 1000 nodes, and rectified linear activation function
    my_model = Dense(1024, activation='relu')(my_model)  # Layer #2
    my_model = Dense(512, activation='relu')(my_model)
    final_layer = Dense(DATASET_CATEGORIES, activation='softmax')(
        my_model)  # DATASET_CATEGORIES because it could be A,B,C etc.
    final_model = Model(inputs=pre_trained_base_model.input, outputs=final_layer)

    #print(final_model.summary())
    return final_model

In [0]:
def freezeLayers(numOfLayers, model):
    for single_layer in model.layers[:numOfLayers]:
        single_layer.trainable = False
    for single_layer in model.layers[numOfLayers:]:
        single_layer.trainable = True

    return model

In [0]:
def fitModel(model):
    model.compile(optimizer='Adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Checkpoints: save only the best performing model weights as file
    checkpoint = ModelCheckpoint('/content/output/model-weights-best.hdf5',
                                 monitor='val_accuracy', verbose=1,
                                 save_best_only=True,
                                 save_weights_only=False, mode='max')
    callbacks_list = [checkpoint]

    step_size_training = (train_generator.n // train_generator.batch_size) * 0.7 # smart to use a small learning rate because
    # high learning rates increase the risk of losing previous knowledge.# Balance here to find optimal step_size
    history = model.fit_generator(generator=train_generator,
                                  steps_per_epoch=step_size_training,
                                  epochs=EPOCHS,
                                  validation_data=validation_generator,
                                  validation_steps=validation_generator.n // BATCH_SIZE,
                                  callbacks=callbacks_list,
                                  )

    # # Save training history as file. For ex. loss and accuracy in each epoch
    # with open('./output/history.json', 'w') as file:
    #     json.dump(history.history, file)

    return model

In [0]:
def trainNetwork():
    preTrainedModel = getPreTrainedModel()
    trimmedModel = freezeLayers(20, preTrainedModel)
    finalModel = fitModel(trimmedModel)
    saveModelToJson(finalModel)
    return finalModel

In [0]:
def saveModelToJson(model):
    model_json = model.to_json()
    with open("/content/output/model.json", "w") as json_file:
        json_file.write(model_json)

    print("Model saved as /content/output/model.json")
    #model.save_weights("./output/model-weights.h5")
    #print("Model Weights saved as ./output/model.h5")


def loadModelfromJson(modelPath, weightPath):
    json_file = open(modelPath, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightPath)
    print("Model loaded from", modelPath)
    return loaded_model


In [0]:
def loadSingleImage(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')
    np_image = transform.resize(np_image, (IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image


def showImageCV(preds, imagePath, letter):
    # find the class label index with the largest corresponding
    # probability
    i = preds.argmax(axis=1)[0]
    # label = lb.classes_[i]
    # draw the class label + probability on the output image
    text = "{}: {:.2f}%".format(letter, preds[0][i] * 100)

    image = cv2.imread(imagePath)
    output = image.copy()
    cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                (0, 0, 255), 2)
    # show the output image
    cv2_imshow(output)
    cv2.waitKey(0)


In [0]:
def getTopPredictions(preds):
    # top_preds = preds.argmax(axis=1)[0]

    predsDict = {
        'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0,
        'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 
        'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0,
        'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0, 'Å': 0, 
        'Ä': 0, 'Ö': 0, 
    }
    # map preds index with probability to correct letter from dictonary
    for i, key in enumerate(predsDict, start=0):
        predsDict[key] = preds[i]

    # sort by dictonary value and returns as list
    all_preds = sorted(predsDict.items(), reverse=True, key=lambda x: x[1])

    top_preds = [all_preds[0], all_preds[1], all_preds[2]]
    # for i, key in enumerate(all_preds, start=0):
    #     top_preds.append(

    # print(all_preds.keys()[0])
    # Convert from float into percentages
    # top_preds = pd.DataFrame(preds / float(np.sum(preds))).applymap(lambda preds: '{:.2%}'.format(preds)).values
    # print(preds[0])

    return top_preds, all_preds

In [0]:
def evaluateModel(model):

    model.compile(optimizer='Adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    score = model.evaluate_generator(validation_generator,
                                     validation_generator.n / BATCH_SIZE,
                                     verbose=1)
    loss, accuracy = score[0], score[1]
    return loss, accuracy


In [0]:
def mainPipeline():
    finalModel = trainNetwork()
    #finalModel = loadModelfromJson('/content/output/model.json', '/content/output/model-weights-best.hdf5')

    #letter = 'C'
    #imagePath = '/content/SSL-Dataset/train' + letter + '/' + letter + '575.jpg'
    imagePath = '/content/kex-dataset/test-images/S/S512.jpg'


    filenames = test_generator.filenames
    letter = filenames[0][0]
    predictions = finalModel.predict_generator(test_generator,steps = len(filenames))
    #image = loadSingleImage(imagePath)
    #predictions = finalModel.predict(image)

    print('*****************************************************')
    #print(predictions)
    #loss, accuracy = evaluateModel(finalModel)
    #print("Loss: ", loss, "Accuracy: ", accuracy * 100, '%')
    print('Input was:', letter)
    top_three_preds, all_preds = getTopPredictions(predictions[0])
    print(top_three_preds)
    # print(predictions)
    #files.download('/content/output/model-weights-best.hdf5')
    #files.download('/content/output/model.json')

    print('*****************************************************')
    # print(finalModel.summary())
    # print(evaluateModel(finalModel))

    showImageCV(predictions, imagePath, letter)

    # Output updated training data structure in text-file
    #os.system("tree --filelimit=20 > project-file-structure.txt")

In [0]:
mainPipeline()

Epoch 1/10
318/317 [==============================] - ETA: 0s - loss: 0.7808 - accuracy: 0.7750
Epoch 00001: val_accuracy improved from -inf to 0.71042, saving model to /content/output/model-weights-best.hdf5
318/317 [==============================] - 182s 573ms/step - loss: 0.7808 - accuracy: 0.7750 - val_loss: 0.8681 - val_accuracy: 0.7104
Epoch 2/10
 76/317 [======>.......................] - ETA: 2:06 - loss: 0.2517 - accuracy: 0.9317